In [ ]:

import SimpleITK as sitk 
from tqdm import tqdm
import matplotlib.pyplot as plt
%pylab inline

import os
import torch
from PIL import Image
from torch.utils import data
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torchvision
import random
from keras.utils import to_categorical

In [ ]:
test_list = []
train_list = []

for rootname, dirnames, filenames in os.walk('/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/HGG'):
    for dirname in dirnames:
        train_list.append(os.path.join(rootname, dirname))
for rootname, dirnames, filenames in os.walk('/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/LGG'):
    for dirname in dirnames:
        train_list.append(os.path.join(rootname, dirname))
        
for rootname, dirnames, filenames in os.walk('/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/test/HGG_LGG'):
    for dirname in dirnames:
        test_list.append(os.path.join(rootname, dirname))

In [ ]:
os.mkdir('/kaggle/working/features/')
os.mkdir('/kaggle/working/labels/')

In [ ]:
len(train_list), len(test_list)

In [ ]:
def load_imgs(path, idx):
    path = path + "/"
    my_dir = sorted(os.listdir(path))
    imgs = []
    print("{} start".format(path))
    for i in range(len(my_dir)):
        if i != len(my_dir)-1:
#         if i == 0:
            img_itk = sitk.ReadImage(path + my_dir[i])
            # pre-processing
            # Image Smoothing/Denoising
            img_itk = sitk.CurvatureFlow(image1=img_itk,timeStep=0.125, numberOfIterations=5)
            img = sitk.GetArrayFromImage(img_itk)
            img = np.asarray(img,dtype=np.float32)
            img = img[30:120,30:222,30:222]
            imgs.append(img)
#             features_list.append(img)
        else:
            img_itk = sitk.ReadImage(path + my_dir[i])
            mask = sitk.GetArrayFromImage(img_itk)
            mask = np.asarray(mask,dtype=np.float32)
            mask = mask[30:120,30:222,30:222]
#             labels_list.append(mask)
    
    imgs = np.asarray(imgs,dtype=np.float32)
    imgs = np.moveaxis(imgs, 0, 1)
    mask = np.asarray(mask,dtype=np.uint8)
    mask = mask[: , None, :, :]

    np.save('/kaggle/working/features/'+str(idx)+'_data.npy',imgs)
    np.save('/kaggle/working/labels/'+str(idx)+'_gt.npy',mask)
#     labels_list = np.concatenate((labels_list, mask))
    print(imgs.shape, mask.shape)
    print(str(idx)+" dir done!")
#     print(len(features_list), len(labels_list))

#     return None

In [ ]:
idx = 0
for dir_path in train_list:
    idx += 1
    load_imgs(dir_path, idx)

In [ ]:
idx

In [ ]:
features_list = np.asarray(features_list,dtype=np.float32)
labels_list = np.asarray(labels_list,dtype=np.float32)
print(features_list.shape, labels_list.shape)
new_features_list = features_list.reshape((-1, 4, 240, 240))
new_labels_list = labels_list.reshape((-1, 1, 240, 240))
print(new_features_list.shape, new_labels_list.shape)

In [ ]:
imgs_shape, mask_shape = load_imgs(train_list[0])
imgs_shape, mask_shape

In [ ]:
def load_data(path):
    path = path + "/"
    my_dir = sorted(os.listdir(path))
    data = []
    gt = []
#     print(my_dir)
    for i in range(len(my_dir)):
        if i != len(my_dir)-1:
            img_itk = sitk.ReadImage(path + my_dir[i])
            # pre-processing
            # Image Smoothing/Denoising
            img_itk = sitk.CurvatureFlow(image1=img_itk,timeStep=0.125, numberOfIterations=5)
            img = sitk.GetArrayFromImage(img_itk)
            data.append(img)
        else:
            img_itk = sitk.ReadImage(path + my_dir[i])
            mask = sitk.GetArrayFromImage(img_itk)
            gt.append(mask)
            
    data = np.asarray(data,dtype=np.float32)
    gt = np.asarray(gt,dtype=np.uint8)
    
    return data, gt


In [ ]:
def sitk_show(img, title=None, margin=0.0, dpi=40):
    nda = sitk.GetArrayFromImage(img)
#     nda=numpy.transpose(nda)
#     print(nda.shape)
    #spacing = img.GetSpacing()
    figsize = (1 + margin) * nda.shape[0] / dpi, (1 + margin) * nda.shape[1] / dpi
    #extent = (0, nda.shape[1]*spacing[1], nda.shape[0]*spacing[0], 0)
    extent = (0, nda.shape[1], nda.shape[0], 0)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

    plt.set_cmap("gray")
    ax.imshow(nda,extent=extent,interpolation=None)
    print(nda.shape)
    if title:
        plt.title(title)
    
    plt.show()

In [ ]:
filenameTC="/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/HGG/brats_2013_pat0013_1/VSD.Brain.XX.O.MR_T1c.54586.mha"
# Slice index to visualize with 'sitk_show'
label="/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/HGG/brats_2013_pat0013_1/VSD.Brain_3more.XX.O.OT.54589.mha"
imgTCOriginal = sitk.ReadImage(filenameTC)
labelOrignal=sitk.ReadImage(label)
print(type(imgTCOriginal))
sitk_show(imgTCOriginal[:, :, 105])
sitk_show(labelOrignal[:, :, 105])

In [ ]:
path1 = "/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/HGG/"
path2 = "/kaggle/input/brain-tumor-segmentation-in-mri-brats-2015/train/LGG/"
# data1, gt1 = load_data(path1)
data2, gt2 = load_data(train_list[1])


In [ ]:
data2.shape,gt2.shape,data2.dtype,gt2.dtype

In [ ]:
class BraTsDataset(Dataset):
    def __init__(self, transform=None,train=True):
        
        self.transform = transform
        self.train = train
        if self.train:
            self.path_list = train_list
            self.data_len = len(train_list)
        else:
            self.path_list = test_list
            self.data_len = len(test_list)
            
    def __getitem__(self, i):
        # read data as ndarray
        features, label = load_data(self.path_list[i])
        # pre-processing -2
#         print(i,"load")
        '''Taking Only from 30th slice till 120th slice for creating the data and also cropping 
        the images to discard useless background 
        So, from 240x240x4 each image now becomes of shape 192x192x4 taking all 4 modalities'''
#         (4, 90, 240, 240)
#         (1, 90, 240, 240)
        features = features[:,43:107,30:222,30:222]
        label = label[:,43:107,30:222,30:222]
#         print("features shape: {}, label shape: {}".format(features.shape, label.shape))
#         features = features[:,30:120,30:222,30:222].reshape([-1,192,192,4])
#         label = label[:,30:120,30:222,30:222].reshape([-1,192,192,1])
#         image1 = image.transpose(3,0)
#         image1 = image1.transpose(3,1)
#         image1 = image1.transpose(3,2)
        print(features.shape, label.shape)
        
        '''The segmentations are combined to generate the final labels of the tumor structures:
        0 background
        necrotic/cystic core (綠) (標註為1), 
        edema (黃) (標註為2), 
        non-enhancing solid core (紅) (標註為3)
        , enhancing core (藍) (標註為4).'''
        
        # one-hot encoding
#         label = np.eye(4)[label]
        label = to_categorical(label)

        
        if self.transform:
            features = self.transform(features)
            label = self.transform(label)
        
        features = torch.from_numpy(features)
        label = torch.from_numpy(label)
#         print("label shape {}".format(label.shape))
#         print("label type {}".format(type(label)))
        label = label.transpose(4, 1)
#         print("label shape {}".format(label.shape))
        label = label.transpose(4, 2)
#         print("label shape {}".format(label.shape))
        label = label.transpose(4, 3)
#         print("label shape {}".format(label.shape))
#         print(i,"from_numpy")
        print(features.shape, label.shape)
        return features, label
    
    def __len__(self):
        return self.data_len